# Project Top Spin
### CIS 5500: Database & Information Systems<br>University of Pennsylvania<br>Fall 2023
Members:
* Peter Akioyamen
* Akanksha Ashok
* Noah Capp 
* Bilal Ali (bilalali@seas.upenn.edu; @bilala45)

## Data Aggregation & Preliminary Cleaning
In this notebook we take all the tennis matches that are split across various files, and aggregate them. Clean the data, addressing anomolies and duplicates, and then split the resulting aggregate data into CSVs which will be loaded into the MySQL database.

In [14]:
# import relevant libraries
import os
import pandas as pd
import numpy as np

stat_cols = [
    "w_ace",
    "w_df",
    "w_svpt",
    "w_1stIn",
    "w_1stWon",
    "w_2ndWon",
    "w_SvGms",
    "w_bpSaved",
    "w_bpFaced",
    "l_ace",
    "l_df",
    "l_svpt",
    "l_1stIn",
    "l_1stWon",
    "l_2ndWon",
    "l_SvGms",
    "l_bpSaved",
    "l_bpFaced"
]

### ATP Data

In [2]:
# get file path and load each file into a dataframe
atp_path = "../../../Data/tennis_atp-master/Data"
atp_files = os.listdir(atp_path)
atp_dfs = []
for file in atp_files:
    df = pd.read_csv(atp_path + "/" + file)
    df["tourney_id"] = df["tourney_id"] + "-Futures" if "futures" in file else df["tourney_id"] + "-Challenger" if "qual_chall" in file else df["tourney_id"] + "-World"
    atp_dfs.append(df)

# combine dataframes
atp_df = pd.concat(atp_dfs, ignore_index=True)

print(f"Data has shape: {atp_df.shape}")
atp_df.describe()

/var/folders/2r/46yljtfx7td9ss61ym7kwqdr0000gn/T/ipykernel_95615/3026338498.py:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(atp_path + "/" + file)


Data has shape: (899360, 49)


,tourney_date,match_num,winner_id,winner_seed,winner_ht,winner_age,loser_id,loser_seed,loser_ht,loser_age,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
count,8.993600e+05,899360.000000,899360.000000,379493.000000,547499.000000,886153.000000,899360.000000,194337.000000,438318.000000,871300.000000,...,196698.000000,196698.000000,196698.000000,196697.000000,196698.000000,196698.000000,792214.000000,723916.000000,726336.000000,666642.000000
mean,2.002999e+07,78.751169,112209.210858,4.823857,184.192344,24.029987,114660.868900,5.571126,184.131211,23.650991,...,45.531902,29.729224,13.832647,11.460805,4.629015,8.525186,445.168236,306.512280,555.666379,215.258565
std,1.723395e+05,156.885405,24920.649989,3.738084,6.514235,3.973687,27323.650455,4.047273,6.524508,4.193583,...,17.322196,12.987956,6.640398,3.687003,3.128239,3.850807,369.205850,816.304278,452.259853,501.422084
min,1.877071e+07,1.000000,100001.000000,1.000000,145.000000,14.000000,100001.000000,1.000000,145.000000,14.000000,...,0.000000,0.000000,0.000000,0.000000,-6.000000,0.000000,1.000000,0.000000,1.000000,0.000000
25%,1.996061e+07,10.000000,102497.000000,2.000000,180.000000,21.100000,102750.000000,3.000000,180.000000,20.500000,...,33.000000,20.000000,9.000000,9.000000,2.000000,6.000000,149.000000,27.000000,179.000000,12.000000
50%,2.007012e+07,21.000000,104354.000000,4.000000,185.000000,23.600000,104721.000000,5.000000,185.000000,23.100000,...,43.000000,28.000000,13.000000,10.000000,4.000000,8.000000,372.000000,83.000000,451.000000,56.000000
75%,2.015051e+07,49.000000,106078.000000,7.000000,188.000000,26.400000,109771.000000,7.000000,188.000000,26.100000,...,55.000000,37.000000,18.000000,14.000000,6.000000,11.000000,635.000000,249.000000,832.000000,211.000000
max,2.023090e+07,7316.000000,212464.000000,35.000000,211.000000,100.700000,212464.000000,38.000000,211.000000,100.800000,...,328.000000,284.000000,101.000000,91.000000,28.000000,38.000000,2257.000000,16950.000000,2267.000000,16950.000000


(191303, 49)

In [3]:
atp_df.describe().filter(stat_cols)

,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced
count,196699.000000,196698.000000,196698.000000,196698.000000,196698.000000,196698.000000,196697.000000,196698.000000,196698.000000,196698.000000,196699.000000,196698.000000,196698.000000,196698.000000,196698.000000,196697.000000,196698.000000,196698.000000
mean,5.683913,2.647856,73.502273,45.132767,33.671908,15.551490,11.641255,3.409679,5.028302,4.203607,3.421187,76.173591,45.531902,29.729224,13.832647,11.460805,4.629015,8.525186
std,4.762690,2.263040,26.438810,17.254571,12.174137,6.377246,3.680094,2.979534,3.911234,4.164408,2.509094,26.124724,17.322196,12.987956,6.640398,3.687003,3.128239,3.850807
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-6.000000,0.000000
25%,2.000000,1.000000,54.000000,33.000000,25.000000,11.000000,9.000000,1.000000,2.000000,1.000000,2.000000,57.000000,33.000000,20.000000,9.000000,9.000000,2.000000,6.000000
50%,5.000000,2.000000,69.000000,42.000000,32.000000,15.000000,11.000000,3.000000,4.000000,3.000000,3.000000,71.000000,43.000000,28.000000,13.000000,10.000000,4.000000,8.000000
75%,8.000000,4.000000,89.000000,55.000000,40.000000,19.000000,14.000000,5.000000,7.000000,6.000000,5.000000,91.000000,55.000000,37.000000,18.000000,14.000000,6.000000,11.000000
max,113.000000,26.000000,491.000000,361.000000,292.000000,82.000000,90.000000,24.000000,34.000000,103.000000,26.000000,489.000000,328.000000,284.000000,101.000000,91.000000,28.000000,38.000000


In [6]:
# get all unique players that were winners and losers
unique_atp_players = pd.concat(
    [
        atp_df.filter([
            "loser_id",
            "loser_name",
            "loser_hand",
            "loser_ht",
            "loser_ioc",
        ], axis=1).rename({
            "loser_id": "id",
            "loser_name": "name",
            "loser_hand": "hand",
            "loser_ht": "height",
            "loser_ioc": "ioc",
        }, axis=1),
        atp_df.filter([
            "winner_id",
            "winner_name",
            "winner_hand",
            "winner_ht",
            "winner_ioc",
        ], axis=1).rename({
            "winner_id": "id",
            "winner_name": "name",
            "winner_hand": "hand",
            "winner_ht": "height",
            "winner_ioc": "ioc",
        }, axis=1),
    ]
).drop_duplicates()

# create new index for atp players
unique_atp_players = unique_atp_players\
    .reset_index(drop=True)\
        .reset_index(drop=False)

# load players data to get dob info
dob_df = pd.read_csv("../../../Data/tennis_atp-master/atp_players.csv")[["player_id", "dob"]]

# merge player info with dob info
unique_atp_players = unique_atp_players.merge(
    dob_df,
    how="inner",
    left_on="id",
    right_on="player_id"
).drop("player_id", axis=1)

# replace player ids
atp_updated_df = atp_df.copy(deep=True)
atp_updated_df["winner_id"] = atp_updated_df["winner_id"].replace(
    to_replace=dict(zip(unique_atp_players["id"], unique_atp_players["index"]))
)
atp_updated_df["loser_id"] = atp_updated_df["loser_id"].replace(
    to_replace=dict(zip(unique_atp_players["id"], unique_atp_players["index"]))
)

# reset id of players
unique_atp_players["id"] = unique_atp_players["index"]
unique_atp_players = unique_atp_players.drop("index", axis=1)


atp_updated_df

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2019-M020-World,Brisbane,Hard,32,A,20181231,300,111,2.0,NaN,...,54.0,34.0,20.0,14.0,10.0,15.0,9.0,3590.0,16.0,1977.0
1,2019-M020-World,Brisbane,Hard,32,A,20181231,299,0,4.0,NaN,...,52.0,36.0,7.0,10.0,10.0,13.0,16.0,1977.0,239.0,200.0
2,2019-M020-World,Brisbane,Hard,32,A,20181231,298,111,2.0,NaN,...,27.0,15.0,6.0,8.0,1.0,5.0,9.0,3590.0,40.0,1050.0
3,2019-M020-World,Brisbane,Hard,32,A,20181231,297,1,NaN,PR,...,60.0,38.0,9.0,11.0,4.0,6.0,239.0,200.0,31.0,1298.0
4,2019-M020-World,Brisbane,Hard,32,A,20181231,296,0,4.0,NaN,...,56.0,46.0,19.0,15.0,2.0,4.0,16.0,1977.0,18.0,1855.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899355,2014-605-World,Tour Finals,Hard,8,F,20141109,514,28,1.0,NaN,...,38.0,29.0,5.0,11.0,3.0,9.0,1.0,10010.0,5.0,4625.0
899356,2014-605-World,Tour Finals,Hard,8,F,20141109,515,28,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,10010.0,2.0,8700.0
899357,2014-D015-World,Davis Cup WG F: FRA vs SUI,Clay,4,D,20141121,1,31,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,5295.0,12.0,2740.0
899358,2014-D015-World,Davis Cup WG F: FRA vs SUI,Clay,4,D,20141121,2,114,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,19.0,1825.0,2.0,9700.0


### WTA Data

In [9]:
# get file path and load each file into a dataframe
wta_path = "../../../Data/tennis_wta-master/Data"
wta_files = os.listdir(wta_path)
wta_dfs = []
for file in wta_files:
    df = pd.read_csv(wta_path + "/" + file, encoding = "ISO-8859-1")
    wta_dfs.append(df)

# combine dataframes
wta_df = pd.concat(wta_dfs, ignore_index=True).iloc[:,:-1]

print(f"Data has shape: {wta_df.shape}")
wta_df.describe()

Data has shape: (154870, 49)


,tourney_date,match_num,winner_id,winner_ht,winner_age,loser_id,loser_ht,loser_age,best_of,minutes,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
count,1.548700e+05,154870.000000,154870.000000,114233.000000,152196.000000,154870.000000,97412.000000,147263.000000,154870.000000,19133.000000,...,40852.000000,40852.000000,40852.000000,21007.000000,40850.000000,40850.000000,103255.000000,97910.000000,99599.000000,95418.000000
mean,1.995096e+07,54.436263,201905.924879,172.316537,23.565427,202908.215574,171.906890,23.488889,3.000284,98.362672,...,43.602712,25.122050,10.967688,10.322416,4.915667,9.954027,77.503956,1134.436850,106.512184,747.150349
std,1.600330e+05,118.363591,4193.028691,6.770742,4.336628,5908.327820,6.728992,4.381044,0.076401,40.018852,...,16.247037,10.834857,5.415292,3.294658,3.152767,3.905101,116.602793,1555.196562,135.028003,1050.340992
min,1.967122e+07,1.000000,113190.000000,146.000000,-16.500000,113190.000000,146.000000,-28.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000
25%,1.981063e+07,8.000000,200274.000000,168.000000,20.300000,200378.000000,168.000000,20.300000,3.000000,73.000000,...,32.000000,17.000000,7.000000,8.000000,3.000000,7.000000,17.000000,78.000000,36.000000,38.000000
50%,1.995081e+07,20.000000,200681.000000,173.000000,23.200000,200923.000000,172.000000,23.000000,3.000000,92.000000,...,41.000000,24.000000,10.000000,10.000000,4.000000,10.000000,44.000000,616.000000,70.000000,439.500000
75%,2.009030e+07,45.000000,201466.000000,176.000000,26.300000,201565.000000,176.000000,26.200000,3.000000,121.000000,...,53.000000,32.000000,14.000000,13.000000,7.000000,12.000000,90.000000,1448.000000,120.000000,968.000000
max,2.023083e+07,2701.000000,263853.000000,190.000000,57.000000,263852.000000,190.000000,67.400000,5.000000,2475.000000,...,774.000000,369.000000,74.000000,24.000000,81.000000,180.000000,1503.000000,13615.000000,1556.000000,13615.000000


In [10]:
# get rid of early rounds that create non-uniqueness between tourney_id and match_num (these matchs are unimportant and can be dropped)
wta_df = wta_df.loc[~wta_df.index.isin([114412, 18561, 18562, 18563, 18564, 18565, 18566, 18567, 18568, 138953, 138954, 138955, 111737, 84588, 84590, 84592, 33532, 33534, 33536])]

In [12]:
# get all unique players that were winners and losers
unique_wta_players = pd.concat(
    [
        wta_df.filter([
            "loser_id",
            "loser_name",
            "loser_hand",
            "loser_ht",
            "loser_ioc",
        ], axis=1).rename({
            "loser_id": "id",
            "loser_name": "name",
            "loser_hand": "hand",
            "loser_ht": "height",
            "loser_ioc": "ioc",
        }, axis=1),
        wta_df.filter([
            "winner_id",
            "winner_name",
            "winner_hand",
            "winner_ht",
            "winner_ioc",
        ], axis=1).rename({
            "winner_id": "id",
            "winner_name": "name",
            "winner_hand": "hand",
            "winner_ht": "height",
            "winner_ioc": "ioc",
        }, axis=1),
    ]
).drop_duplicates()

# create new index for wta players
unique_wta_players = unique_wta_players\
    .reset_index(drop=True)\
        .reset_index(drop=False)
unique_wta_players["index"] = unique_wta_players["index"] + 1 + unique_atp_players.id.max()

# load players data to get dob info
wta_athletes = pd.read_csv("../../../Data/tennis_wta-master/wta_players.csv")[["player_id", "dob"]]

# merge player info with dob info
unique_wta_players = unique_wta_players.merge(
    wta_athletes,
    how="inner",
    left_on="id",
    right_on="player_id"
).drop("player_id", axis=1)

# replace player ids
wta_updated_df = wta_df.copy(deep=True)
wta_updated_df["winner_id"] = wta_updated_df["winner_id"].replace(
    to_replace=dict(zip(unique_wta_players["id"], unique_wta_players["index"]))
)
wta_updated_df["loser_id"] = wta_updated_df["loser_id"].replace(
    to_replace=dict(zip(unique_wta_players["id"], unique_wta_players["index"]))
)

# reset id of players
unique_wta_players["id"] = unique_wta_players["index"]
unique_wta_players = unique_wta_players.drop("index", axis=1)

wta_updated_df

/var/folders/2r/46yljtfx7td9ss61ym7kwqdr0000gn/T/ipykernel_95615/1457411198.py:40: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  wta_athletes = pd.read_csv("../../../Data/tennis_wta-master/wta_players.csv")[["player_id", "dob"]]


,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,1975-W-SL-AUS-01A-1975,Australian Open,Grass,56.0,G,19741221.0,1.0,33534.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1975-W-SL-AUS-01A-1975,Australian Open,Grass,56.0,G,19741221.0,2.0,33535.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1975-W-SL-AUS-01A-1975,Australian Open,Grass,56.0,G,19741221.0,3.0,33550.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1975-W-SL-AUS-01A-1975,Australian Open,Grass,56.0,G,19741221.0,4.0,33536.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1975-W-SL-AUS-01A-1975,Australian Open,Grass,56.0,G,19741221.0,5.0,33551.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154865,1978-W001,Wightman Cup,Carpet,NaN,D,19781103.0,1.0,33657.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154866,1978-W001,Wightman Cup,Carpet,NaN,D,19781103.0,2.0,33648.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154867,1978-W001,Wightman Cup,Carpet,NaN,D,19781103.0,3.0,33656.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154868,1978-W001,Wightman Cup,Carpet,NaN,D,19781103.0,4.0,33657.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# convert date columns to appropriate datetime formats
atp_updated_df["tourney_date"] = pd.to_datetime(
    atp_updated_df.tourney_date, 
    format="%Y%m%d"
)
wta_updated_df["tourney_date"] = pd.to_datetime(
    wta_updated_df.tourney_date, 
    format="%Y%m%d"
)
unique_atp_players["dob"] = pd.to_datetime(
    unique_atp_players.dob, 
    format="%Y%m%d",
    errors="coerce"
)
unique_wta_players["dob"] = pd.to_datetime(
    unique_wta_players.dob, 
    format="%Y%m%d",
    errors="coerce"
)

# format draw_size as a string
atp_updated_df["draw_size"] = atp_updated_df.draw_size.apply(lambda x: str(int(x)) if str(x).replace(".", "", 1).isdigit() else x)
wta_updated_df = wta_updated_df[wta_updated_df.draw_size != "exho"]
wta_updated_df["draw_size"] = wta_updated_df.draw_size.apply(lambda x: str(int(x)) if str(x).replace(".", "", 1).isdigit() else x)

# keep the appropriate version/accurate version of the duplicate players in the data
unique_atp_players = unique_atp_players[~unique_atp_players.duplicated(subset="id", keep="last")]

# fix data encoding issues
wta_updated_df["winner_id"] = wta_updated_df["winner_id"].astype('Int64')
wta_updated_df["loser_id"] = wta_updated_df["loser_id"].astype('Int64')
wta_updated_df["draw_size"] = np.floor(pd.to_numeric(wta_updated_df["draw_size"], errors='coerce')).astype('Int64')
wta_updated_df["match_num"] = wta_updated_df["match_num"].astype('Int64')

# set the league of the player and the tournament
unique_atp_players["league"] = "atp"
unique_wta_players["league"] = "wta"
atp_updated_df.loc[:,"league"] = "atp"
wta_updated_df.loc[:,"league"] = "wta"

/var/folders/2r/46yljtfx7td9ss61ym7kwqdr0000gn/T/ipykernel_95615/1998249558.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wta_updated_df["tourney_date"] = pd.to_datetime(


In [16]:
# get complete list of tennis matches and players
tennis_players_complete = pd.concat([unique_atp_players, unique_wta_players], ignore_index=True)
tennis_matches_complete = pd.concat([atp_updated_df, wta_updated_df], ignore_index=True)

In [28]:
# save cleaned data
atp_updated_df.to_csv(
    "../../../Data/FinalData/Aggregated_Tennis_Data/LeagueData/atp_competitions.csv",
    encoding="utf-8",
    index=False
)
wta_updated_df.to_csv(
    "../../../Data/FinalData/Aggregated_Tennis_Data/LeagueData/wta_matches.csv",
    encoding="utf-8",
    index=False
)
unique_atp_players.to_csv(
    "../../../Data/FinalData/Aggregated_Tennis_Data/LeagueData/atp_players.csv",
    encoding="utf-8",
    index=False
)
unique_wta_players.to_csv(
    "../../../Data/FinalData/Aggregated_Tennis_Data/LeagueData/wta_players.csv",
    encoding="utf-8",
    index=False
)
tennis_players_complete.to_csv(
    "../../../Data/FinalData/Aggregated_Tennis_Data/tennis_players_aggregated.csv",
    encoding="utf-8",
    index=False
)
tennis_matches_complete.to_csv(
    "../../../Data/FinalData/Aggregated_Tennis_Data/tennis_competitions_aggregated.csv",
    encoding="utf-8",
    index=False
)

In [33]:
# create tennis tournaments table for db
tennis_tournaments_table = tennis_matches_complete.filter(
    [
        "tourney_id",
        "tourney_name",
        "tourney_date",
        "surface",
        "draw_size",
        "tourney_level",
        "league"

    ]
).drop_duplicates()

# create tennis matches table for db
tennis_matches_table = tennis_matches_complete.drop(
    [
        "tourney_name",
        "tourney_date",
        "surface",
        "draw_size",
        "tourney_level",
        "league"

    ],
    axis=1
)

# save database tables
tennis_tournaments_table.to_csv(
    "../../../Data/FinalData/DB_Tables/tennis_tournaments_table.csv",
    encoding="utf-8",
    index=False
)
tennis_matches_table.to_csv(
    "../../../Data/FinalData/DB_Tables/tennis_matches_table.csv",
    encoding="utf-8",
    index=False
)